In [81]:
import pandas as pd
import sqlite3 as slite
from sqlite3 import Error

In [82]:
def query10(conn):
    '''
        Q10: Produce a list of facilities with a total revenue less than 1000.
        The output of facility name and total revenue, sorted by revenue. Remember
        that there's a different cost for guests and members!
    '''

    print ('''

        Q10: Produce a list of facilities with a total revenue less than 1000.
        The output of facility name and total revenue, sorted by revenue. Remember
        that there's a different cost for guests and members!

        ''')

    cur = conn.cursor()
    cur.execute('''SELECT
                              f.name as facilityname
                             , sum(case when b.memid = 0 then guestcost * slots
                                 else membercost * slots end ) total_cost
                      FROM  bookings b
                     INNER JOIN facilities f
                        ON b.facid = f.facid
                    group by f.name
                    having total_cost < 1000
                    order by total_cost desc'''
                )
    df = pd.DataFrame(cur.fetchall())
    col_nms = [nm[0] for nm in cur.description]
    df.columns = col_nms
    print (df)


In [83]:
def query11(conn):
    '''
        Q11: Produce a report of members and who
        recommended them in alphabetic surname,firstname order
    '''
    print ('''

        Q11: Produce a report of members and who
        recommended them in alphabetic surname,firstname order.

        ''')
    cur = conn.cursor()
    cur.execute('''select
                        m.surname||','||m.firstname as mem_nm
                      , r.surname||','||r.firstname as recomemenders_nm
                  from members m
                  left join members r
                    on m.recommendedby  = r.memid
                order by mem_nm , recomemenders_nm;'''
                )
    df = pd.DataFrame(cur.fetchall())
    col_nms = [nm[0] for nm in cur.description]
    df.columns = col_nms
    print (df)

In [84]:
def query12(conn):
    '''
        Q12: Find the facilities with their usage by member, but not guests
    '''
    print ('''

        Q12: Find the facilities with their usage by member, but not guests

        ''')
    cur = conn.cursor()
    cur.execute('''select  f.name
                         , count(distinct m.memid) member_count
                     from members m
                     join bookings b
                      on b.memid = m.memid
                     join facilities f
                       on f.facid = b.facid
                     where m.memid != 0
                     group by f.name'''
                )
    df = pd.DataFrame(cur.fetchall())
    col_nms = [nm[0] for nm in cur.description]
    df.columns = col_nms
    print (df)

In [85]:
def query13(conn):
    '''
        Q13: Find the facilities usage by month, but not guests
    '''
    print ('''

           Q13: Find the facilities usage by month, but not guests

           ''')
    cur = conn.cursor()
    cur.execute('''select f.name
                        , strftime('%m', starttime) month
                        , count(1)
                     from members m
                     join bookings b
                      on b.memid = m.memid
                     join facilities f
                       on f.facid = b.facid
                     where m.memid != 0
                     group by f.name, month'''
                )
    df = pd.DataFrame(cur.fetchall())
    col_nms = [nm[0] for nm in cur.description]
    df.columns = col_nms
    print (df)

In [86]:
def main():
    dbfile_nm = "sqlite_db_pythonsqlite.db"
    conn = slite.connect(dbfile_nm)
    with conn:
        query10(conn)
        query11(conn)
        query12(conn)
        query13(conn)
if __name__=='__main__':
    main()




        Q10: Produce a list of facilities with a total revenue less than 1000.
        The output of facility name and total revenue, sorted by revenue. Remember
        that there's a different cost for guests and members!

        
    facilityname  total_cost
0     Pool Table         270
1  Snooker Table         240
2   Table Tennis         180


        Q11: Produce a report of members and who
        recommended them in alphabetic surname,firstname order.

        
                     mem_nm   recomemenders_nm
0            Bader,Florence    Stibbons,Ponder
1                Baker,Anne    Stibbons,Ponder
2             Baker,Timothy     Farrell,Jemima
3                Boothe,Tim         Rownam,Tim
4            Butters,Gerald       Smith,Darren
5               Coplin,Joan      Baker,Timothy
6             Crumpet,Erica        Smith,Tracy
7                Dare,Nancy    Joplette,Janice
8             Farrell,David               None
9            Farrell,Jemima               None
10    